## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [36]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [37]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [38]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json
from datetime import datetime

## Download

Retrieve the page

In [39]:
url = "https://itwebservices.placer.ca.gov/coviddashboard/#cases-by-zip-code"

In [40]:
page = requests.get(url)

## Parse

In [41]:
soup = BeautifulSoup(page.content, "html.parser")

Find script tag with the Infogram data

In [42]:
for elem in soup(text=re.compile(r"Foresthill")):
    script_content = elem

In [43]:
data = json.loads(script_content)

In [44]:
zip_cities_list = data["x"]["calls"][1]["args"][-3]

In [45]:
dict_list = []

In [46]:
TAG_RE = re.compile(r"<[^>]+>")

In [47]:
for z in zip_cities_list:
    split_zips = z.split("<br/>")
    area = TAG_RE.sub("", str(split_zips[0]))
    zip_code, city = area.split(" - ")
    clean_cases = re.sub("Number of cases: ".lower(), "", str(split_zips[1]).lower())
    d = dict(area=area, city=city, zip_code=zip_code, confirmed_cases=clean_cases)
    dict_list.append(d)

In [48]:
df = pd.DataFrame(dict_list)

Get timestamp

In [49]:
time_div = soup.find("div", id="dashboard-data-last-updated")

In [50]:
timestamp = time_div.find("span", class_="value-output").get_text()

In [51]:
latest_date = pd.to_datetime(timestamp).date()

In [52]:
df["county_date"] = latest_date

In [53]:
df.insert(0, "county", "Placer")

Clean up

In [54]:
df["area"] = df["area"].str.replace(" - ", ": ")

In [55]:
df = df[["county", "area", "confirmed_cases", "county_date", "zip_code"]].rename(
    columns={"zip_code": "zip"}
)

## Vet

In [56]:
try:
    assert not len(df) < 32
except AssertionError:
    raise AssertionError("Placer County's scraper is missing rows")

In [57]:
try:
    assert not len(df) > 32
except AssertionError:
    raise AssertionError("Placer County's scraper has more rows than before")

## Export

Set date

In [58]:
tz = pytz.timezone("America/Los_Angeles")

In [59]:
today = datetime.now(tz).date()

In [60]:
slug = "placer"

In [61]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [62]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [63]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [64]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [65]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)